# HW2 Aksenova NLP

## Русский текст

Тут есть опасные/спорные места с лексемами из двух слов. С ними вообще ничего не справляется ("то есть", "вряд ли"). Кроме того, есть слово "печь", часть моделей парсят его как инфинитив. Еще есть заимствования типа "глегг", "люсикаттен" и "икея". Есть очень спорный момент с "друг друга". Михаил Александрович считает, что это местоимение, модели с ним не согласны.

In [21]:
ru_eval = """Каждый год в канун католического рождества мы со шведскоговорящими друзьями отправляемся в РГГУ праздновать день шведской святой Люсии. 
            Накануне каждый готовит что-то к празднику. Кто-то решается печь люсикаттен, то есть специальные булочки с шафраном и изюмом, кто-то запасается глеггом из икеи. 
            И потом мы все вместе поем песни, отгадываем загадки и поздравляем друг друга с наступающими праздниками.
            Недавно к нам приезжала Анн Линде – министр иностранных дел Швеции, как настоящая фемактивистка она выступила в поддержку российских женщин.
            Но вообще это странно, что в России принято говорить "с наступающим" только про новый год. Вряд ли кто-то знает в точности, как так вышло...  """

In [22]:
words = []
tags = []
with open('ru_eval.txt') as fh:
  for i in fh.readlines():
    i.strip()
    word, tag = i.split()
    words.append(word)
    tags.append(tag)

In [23]:
import pandas as pd 

df = pd.DataFrame(list(zip(words, tags)), 
               columns =['word', 'real']) 
df 

,word,real
0,Каждый,AP
1,год,S
2,в,PR
3,канун,S
4,католического,A
...,...,...
95,в,PR
96,точности,S
97,как,CO
98,так,AD


In [24]:
tags_dict_RU = {"A" : ["ADJF", 'ADJS', 'ADJ', 'A', 'COMP', 'PRED'], 'AP' : ['AP', "DET"], 
             'CO' : ['CONJ', 'CO', 'SCONJ', 'CCONJ'], 
             'NUM' : ['NUM', 'NUMR', 'ANUM'], 'SP' : ['SPRO', 'SP', 'NPRO'], 
             'PR' : ['PR', 'PREP', 'ADP', 'PRON'], 'ADV' : ['ADV', 'ADVPRO', 'ADVB'],
             'V' : ['VERB', 'INFN', 'GRND', 'PRTF', 'PRTS'],
             'PA': ['PART', 'PA', 'PRCL'], 'S' : ['NOUN', 'S', 'NNP', 'PROPN'],
             'INTJ' : ['INTJ'], 'X' : ['FW']}

In [25]:
ru_tags = {}
for key, values in tags_dict_RU.items():
  for value in values:
    ru_tags[value] = key

## MyStem

In [26]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-17 13:41:52--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.241, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-17 13:41:52--  http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)... 5.45.220.13, 2a02:6b8:0:2002::14
Connecting to cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)|5.45.220.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linux3.1 100%[===================>] 

In [27]:
from pymystem3 import Mystem
ms = Mystem()

In [28]:
import re

In [29]:
lemmas = ms.analyze(ru_eval)

In [30]:
words = []
tags_ms = []
for lemma in lemmas:
    if ' ' in lemma['text'] or '\n' in lemma['text']:
        continue
    elif len(lemma) > 1:
        words.append(lemma['text'])
        tags_ms.append(re.search(r'\w\w?', lemma['analysis'][0]['gr']).group())

In [31]:
len(tags)

100

In [32]:
df['mystem'] = tags_ms
df

,word,real,mystem
0,Каждый,AP,AP
1,год,S,S
2,в,PR,PR
3,канун,S,S
4,католического,A,A
...,...,...,...
95,в,PR,PR
96,точности,S,S
97,как,CO,CO
98,так,AD,AD


## Pymorphy

In [33]:
! pip install pymorphy2

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 8.2MB/s 


In [34]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [35]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
tags_pm = []
for word in word_tokenize(ru_eval):
  if morph.parse(word)[0].tag.POS != None:
    tags_pm.append(morph.parse(word)[0].tag.POS)

In [37]:
tags_pm = [ru_tags[tag] for tag in tags_pm]

In [38]:
df['pymorphy'] = tags_pm
df

,word,real,mystem,pymorphy
0,Каждый,AP,AP,A
1,год,S,S,S
2,в,PR,PR,PR
3,канун,S,S,S
4,католического,A,A,A
...,...,...,...,...
95,в,PR,PR,PR
96,точности,S,S,S
97,как,CO,CO,CO
98,так,AD,AD,CO


## Natasha

In [39]:
!pip install natasha

     |████████████████████████████████| 34.4MB 89kB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26100 sha256=d5dec00ca2d255100c958b822995c6c10311122efec4d9ad94d2bff5796afd57
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [40]:
import navec

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

In [41]:
text = """Каждый год в канун католического рождества мы с друзьями-шведами отправляемся в РГГУ праздновать день шведской святой Люсии. 
            Накануне каждый готовит что-то к празднику. Кто-то решается печь люсикаттен, то есть специальные булочки с шафраном и изюмом, кто-то запасается глеггом из икеи. 
            И потом мы все вместе поем песни, отгадываем загадки и поздравляем друг друга с наступающими праздниками.
            Недавно к нам приезжала Анн Линде – министр иностранных дел Швеции, как настоящая фемактивистка она выступила в поддержку российских женщин.
            Но вообще это странно, что в России принято говорить "с наступающим" только про новый год. Вряд ли кто-то знает в точности, как так вышло...  """

In [42]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

In [43]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)


#doc = Doc(text)

tags_nat = []
#doc.tag_morph(morph_tagger)
for word in words:
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
#    doc.tokens
#    doc.sents[i].tokens
    tags_nat.append(doc.tokens[0].pos)

In [44]:
len(tags_nat)

100

In [45]:
tags_nat = [ru_tags[tag] for tag in tags_nat]

In [46]:
df['natasha'] = tags_nat
df

,word,real,mystem,pymorphy,natasha
0,Каждый,AP,AP,A,AP
1,год,S,S,S,S
2,в,PR,PR,PR,PR
3,канун,S,S,S,S
4,католического,A,A,A,A
...,...,...,...,...,...
95,в,PR,PR,PR,PR
96,точности,S,S,S,S
97,как,CO,CO,CO,CO
98,так,AD,AD,CO,ADV


## Accuracy

In [47]:
from sklearn import metrics

In [48]:
metrics.accuracy_score(df['real'], df['pymorphy'])

0.77

In [49]:
metrics.accuracy_score(df['real'], df['natasha'])

0.77

In [50]:
metrics.accuracy_score(df['real'], df['mystem'])

0.95

# Английский текст

Я взяла отрывок из Куприна на английском языке. Тут есть слово kvas, которое явно не встречается в нормальном английском. Модели по-разному парсили слова new-bought и sweet-smelling: мы решили считать их единым токеном. off обычно считается наречием, но в выражении be off – прилагательным. so that – та же ситуация, что и "то есть": лексема из двух токенов.

In [76]:
eng_eval = """A tall young man with a fair moustache and thin hair was in the chair. 
              He was dreaming voluptuously how he would be off in an instant 
              on his new-bought bicycle to the bungalow. He would undress quickly, 
              and without waiting to cool, still bathed in sweat, would fling himself into the clear, 
              cold, sweet-smelling sea. His whole body was enervated and tense, thrilled by the thought. 
              Impatiently moving the papers before him, he spoke in a drowsy voice.
              "So, Joseph Moritzovich will conduct the case of Rubinchik...
              Perhaps there is still a statement to be made on the order of the day?"
              His youngest colleague, a short, stout Karaite, very black and lively, 
              said in a whisper so that every one could hear: 'On the order of the day, the best thing would be iced kvas...'"""

In [77]:
tags_dict_en = {"ADJ" : ["JJ", 'JJR', 'JJS', 'ADJ'], 'CONJ' : ['CCONJ', 'CONJ', 'SCONJ', 'CC', 'CD'], 
             'NUM' : ['NUM'], 'DET' : ['DET', 'DT', 'WDT', 'WP$', 'PDT'], 
             'PRON' : ['PRP', 'PRP$', 'WP', 'PRON'], 
             'ADP' : ['IN', 'PD', 'ADP'], 'ADV' : ['RB', 'RBR', 'RBS', 'WRB', 'ADV'],
             'VERB' : ['MD', 'VB', 'VBD', 'VBG', 'VBN', 'VBZ', 'VBP', 'AUX', 'VERB'],
             'PART': ['TO', 'PR', 'POS', 'EX', 'PART', 'RP'], 
             'NOUN' : ['NN', 'NNS', 'NNP', 'NNPS', 'NOUN', 'PROPN'],
             'INTJ' : ['UH', 'INTJ'], 'X' : ['FW', 'X']}

In [78]:
en_tags = {}
for key, values in tags_dict_en.items():
  for value in values:
    en_tags[value] = key

In [79]:
words = []
tags = []
with open('eng_eval.txt') as fh:
  for i in fh.readlines():
    i.strip()
    word, tag = i.split()
    words.append(word)
    tags.append(tag)

In [80]:
df1 = pd.DataFrame(list(zip(words, tags)), 
               columns =['word', 'real']) 
df1 

,word,real
0,A,DET
1,tall,ADJ
2,young,ADJ
3,man,NOUN
4,with,ADP
...,...,...
134,thing,NOUN
135,would,VERB
136,be,VERB
137,iced,VERB


## Spacy

In [81]:
!pip install spacy

In [82]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [83]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(eng_eval)

In [84]:
tags_sp = []
for i, s in enumerate(doc.sents):
  for t in s:
    if t.pos_ != 'PUNCT' and t.pos_ != 'SPACE':
      tags_sp.append(t.pos_)

In [85]:
tags_sp = [en_tags[tag] for tag in tags_sp]

In [86]:
df1['spacy'] = tags_sp
df1

,word,real,spacy
0,A,DET,DET
1,tall,ADJ,ADJ
2,young,ADJ,ADJ
3,man,NOUN,NOUN
4,with,ADP,ADP
...,...,...,...
134,thing,NOUN,NOUN
135,would,VERB,VERB
136,be,VERB,VERB
137,iced,VERB,VERB


## Flair

In [87]:
! pip install flair

In [88]:
punctuation = """,;:”$%^&*№()_—=+!...?|[]{}\"/<>`~±§«»°12''34567890"""

In [89]:
tags_fl = []
words_fl = []

In [90]:
from flair.data import Sentence
from flair.models import SequenceTagger

sentence = Sentence(eng_eval)

tagger = SequenceTagger.load('pos')

tagger.predict(sentence)
print(sentence.to_tagged_string())
for i in sentence.to_tagged_string().split('> '):
  cur_word = i.split(' <')[0]
  if '-' in cur_word:
    tags_fl.extend([i.split(' <')[1], i.split(' <')[1]])
  if cur_word in words:
    words_fl.append(i.split(' <')[0])
    tags_fl.append(i.split(' <')[1])

2020-10-17 14:01:06,187 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt
A <DT> tall <JJ> young <JJ> man <NN> with <IN> a <DT> fair <JJ> moustache <NN> and <CC> thin <JJ> hair <NN> was <VBD> in <IN> the <DT> chair <NN> . <.> He <PRP> was <VBD> dreaming <VBG> voluptuously <RB> how <WRB> he <PRP> would <MD> be <VB> off <RB> in <IN> an <DT> instant <NN> on <IN> his <PRP$> new-bought <JJ> bicycle <NN> to <IN> the <DT> bungalow <NN> . <.> He <PRP> would <MD> undress <VB> quickly <RB> , <,> and <CC> without <IN> waiting <VBG> to <TO> cool <VB> , <,> still <RB> bathed <VBN> in <IN> sweat <NN> , <,> would <MD> fling <VB> himself <PRP> into <IN> the <DT> clear <JJ> , <,> cold <JJ> , <,> sweet-smelling <JJ> sea <NN> . <.> His <PRP$> whole <JJ> body <NN> was <VBD> enervated <VBN> and <CC> tense <JJ> , <,> thrilled <VBN> by <IN> the <DT> thought <NN> . <,> Impatiently <RB> moving <VBG> the <DT> papers <NNS> before <IN> him <PRP> , <,> he <PRP> spoke <VBD> in <IN> a <DT> drowsy <JJ> voice 

In [91]:
len(tags_fl)

139

In [92]:
tags_fl = [en_tags[tag] for tag in tags_fl]

In [93]:
df1['flair'] = tags_fl
df1

,word,real,spacy,flair
0,A,DET,DET,DET
1,tall,ADJ,ADJ,ADJ
2,young,ADJ,ADJ,ADJ
3,man,NOUN,NOUN,NOUN
4,with,ADP,ADP,ADP
...,...,...,...,...
134,thing,NOUN,NOUN,NOUN
135,would,VERB,VERB,VERB
136,be,VERB,VERB,VERB
137,iced,VERB,VERB,VERB


## NLTK

In [94]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [95]:
tags_nltk = [i[1] for i in nltk.pos_tag(words)]

In [96]:
tags_nltk = [en_tags[tag] for tag in tags_nltk]

In [97]:
df1['nltk'] = tags_nltk
df1

,word,real,spacy,flair,nltk
0,A,DET,DET,DET,DET
1,tall,ADJ,ADJ,ADJ,ADJ
2,young,ADJ,ADJ,ADJ,ADJ
3,man,NOUN,NOUN,NOUN,NOUN
4,with,ADP,ADP,ADP,ADP
...,...,...,...,...,...
134,thing,NOUN,NOUN,NOUN,NOUN
135,would,VERB,VERB,VERB,VERB
136,be,VERB,VERB,VERB,VERB
137,iced,VERB,VERB,VERB,VERB


## Accuracy

In [98]:
metrics.accuracy_score(df1['real'], df1['nltk'])

0.8561151079136691

In [99]:
metrics.accuracy_score(df1['real'], df1['spacy'])

0.9568345323741008

In [100]:
metrics.accuracy_score(df1['real'], df1['flair'])

0.8848920863309353